In [1]:
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
import json

In [2]:
# Dữ liệu nhà đất thuê
try:
    with open('tempdata_thue.json', 'r', encoding='utf-8') as f:
        listdata_thue = json.loads(f.read())
        f.close()
except:
    listdata_thue = {}
try:
    with open('currentPage_thue.txt', 'r') as f:
        currentPage = int(f.read())+1
        f.close()
except:
    currentPage = 1
try:
    with open("projectLinks_thue.txt", "r") as f:
        projectLinks_thue = [i for i in f.read().split('\n')]
        f.close()
except:
    projectLinks_thue = []
try:
    with open("postLinks_thue.txt", "r") as f:
        postLinks_thue = [i for i in f.read().split('\n')]
        f.close()
except:
    postLinks_thue = []

with open("projectLinks_thue.txt", "w") as f:
    f.write("")
    f.close()
with open("projectLinks_thue.txt", "a") as f:
    for i in projectLinks_thue:
        f.write(i+'\n')
    f.close()
    
current = len(listdata_thue)+1

for page in range(currentPage, 600, 1):
    print('page', page)
    response = requests.get("https://houseviet.vn/nha-dat-cho-thue/p{}".format(page))
    posts = BeautifulSoup(response.content).find_all('div', {'class', re.compile('item-list')})
    # posts = BeautifulSoup(response.content, "html.parser").find_all('div', {'class', 'item-list fnowrap'}) or BeautifulSoup(response.content, "html.parser").find_all('div', {'class', 'item-list fnowrap'})
    for post in posts:
        Link_Title_Img = post.find('div', {'class', 'img-item'})
        link = Link_Title_Img.find('a').get('href')
        title = Link_Title_Img.find('a').get('title')
        img = Link_Title_Img.find('img', {'class', 'lazy img-object-cover lazyload blur-up'}).get('src')
        itemContent = post.find('div', {'class', 'item-content'})
        time = itemContent.find('p', {'class', 'time'}).find('span', {'class', 'moment'}).get('data-time')
        price = itemContent.find('p', {'class', 'price'}).find('span', {'class', 'text'}).text
        area = itemContent.find('div', {'class', 'area'}).find('span', {'class', 'text'}).text
        Address = itemContent.find('div', {'class', 'address'}).find_all('a')
        address = Address[0].text+', '+Address[1].text or '' 
        sleep(1)
    #     print(link,title,img, time, price, area, address)
        postResponse = requests.get(link)
        postContent = BeautifulSoup(postResponse.content, "html.parser")
        moreInfo_ = postContent.find('ul', {'class', 'fnowrap fcenter short-detail'})
        with open("postLinks_thue.txt", "a") as f:
            f.write(link+'\n')
            f.close()
        if moreInfo_:
            moreInfo = moreInfo_.find_all('span')
            postId, typePost = moreInfo[0].text.strip(), moreInfo[1].text.strip()
            description = postContent.find('div', {'class', 'description'}).text.replace("Thông tin mô tả:", "")
            imgList = [img.get('data-background') for img in (postContent.find_all('div', {'class', 'swiper-slide swiper-lazy'}))] or []
            info_realestates = postContent.find('div', {'class', 'infomation'}).find('div', {'class', 'info-realestate'}).find_all('span')
            infoRealestate = {info_realestates[i].text.replace(":","").strip():info_realestates[i+1].text.strip() for i in range(0, len(info_realestates), 2)}

        #     print(postId)
        #     print(typePost)
        #     print(description)
        #     print(imgList)
        #     print(infoRealestate)
            projectInfo = postContent.find('div', {'class', 'related-project'})
            projectInfor = {}
            if projectInfo:
                projectDetails = projectInfo.find('div', {'class', 'related-project-detail'})
                projectImg_ = projectInfo.find('div', {'class', 'related-project-img'})
                projectImage = projectImg_.find('img').get('src')
                projectName = projectImg_.find('a').get('title')
                projectLink = projectImg_.find('a').get('href')
                #print(projectImage, projectName, projectLink)
                ownerInvestment, projectArea, legalDocument = [i.text.replace("Chủ đầu tư:", "").replace("Quy mô:", "").replace("Giấy tờ pháp lý:", "").replace("Xem Chi Tiết Dự Án", "").strip() for i in projectDetails.find_all('p')]
                if projectLink not in projectLinks_thue:
                    projectLinks_thue.append(projectLink)
                    with open("projectLinks_thue.txt", "a") as f:
                        f.write(projectLink+'\n')
                        f.close()
                projectInfor = {"name":projectName, "img":projectImage, "link":projectLink, "ownerInvestment":ownerInvestment, "projectArea":projectArea, "legalDocument":legalDocument}
            else:
                projectInfor = {}
            listdata_thue[postId] = {"title":title, "type":typePost, "img":img, "link":link, "description":description, "imgList":imgList, "price":price, "address":address, "area":area, "time":time, "infoRealestate":infoRealestate, "projectInfo":projectInfor}
            print(current, end = ' ')
            print(link)
            current+=1
        else:
            continue
        sleep(1)
    # lưu data và page tiếp theo sau mỗi page
    with open('tempdata_thue.json', 'w', encoding='utf-8') as f:
        json.dump(listdata_thue, f, ensure_ascii=False, indent=4)
        f.close()
    with open('currentPage_thue.txt', 'w') as f:
        f.write(str(page))
        f.close()
    sleep(1)

with open('data_thue.json', 'w', encoding='utf-8') as f:
    json.dump(listdata_thue, f, ensure_ascii=False, indent=4)
    f.close()

page 516
6647 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-so-30-6/cho-thue-can-goc-cc-4s2-linh-dong-75m2-gia-chi-6-trieu-thang
6648 https://houseviet.vn/cho-thue-phong-tro-nha-tro-duong-23-8/cho-thue-phong-tro-dep-tai-vi-tri-duong-so-23-kp4-phuong-hiep-binh-chanh-thu-duc-dia-chi-50-7-5
6649 https://houseviet.vn/cho-thue-van-phong-duong-vo-van-ngan/van-phong-cho-thue-quan-thu-duc-dt-20m2-5000m2-gia-160-nghin-m2-lh-18006310-mien-cuoc-goi
6650 https://houseviet.vn/cho-thue-phong-tro-nha-tro-phuong-binh-trung-dong/can-nam-o-ghep-duong-nguyen-thi-dinh-q2-tp-ho-chi-minh-
6651 https://houseviet.vn/cho-thue-phong-tro-nha-tro-pho-le-trong-tan-1/phong-moi-xay-dep-long-lanh-moi-100-tai-so-319-3-le-trong-tan-gia-tu-1-9tr-th-den-2-8tr-th
6652 https://houseviet.vn/cho-thue-phong-tro-nha-tro-duong-khuong-viet-2/chinh-chu-con-3-phong-dep-cho-thue-tai-duong-khuong-viet-cach-dam-sen-chi-5p-di-xe-1-2tr-thang
6653 https://houseviet.vn/cho-thue-phong-tro-nha-tro-duong-son-ky-1/cho-thue-phong-tro-mo

6702 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-xa-lo-ha-noi/chinh-chu-cho-thue-can-ho-dich-vu-mini-noi-that-cao-cap-o-thao-dien-quan-2
page 520
6703 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-xa-lo-ha-noi/chinh-chu-can-cho-thue-can-ho-cao-cap-2-phong-ngu-chung-cu-lexington-residence-view-thoang-dep
6704 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-xa-lo-ha-noi/cho-thue-ban-can-ho-tai-masteri-full-noi-that-gia-re-lh-0938089645-thuy
6705 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-xa-lo-ha-noi/nhung-can-ho-xi-riverview-sang-trong-hien-dai-cho-thue-145m2-185m2-201m2-penthouse
6706 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-xa-lo-ha-noi/minh-cho-thue-cantavil-vi-tri-dep-nhat-quan-2-2-phong-ngu-nha-dep-me-ly-gia-13-trieu-thang
6707 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-xa-lo-ha-noi/cho-thue-can-ho-cao-cap-sarimi-sala-b2-0804-dien-tich-88m2-gia-29-5-trieu-huong-dn-view-cong-vien
6708 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-

6756 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-xa-lo-ha-noi/cho-thue-can-ho-lexington-1pn-2pn-3pn-gia-tot-nhat-thi-truong-lh-0909-642-286-thanh-phat
6757 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-xa-lo-ha-noi/cho-thue-can-ho-cao-cap-imperia-quan-2-noi-that-cao-cap-gia-re-nhat-thi-truong-lh-0909-642-286
6758 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-xa-lo-ha-noi/cho-thue-cc-bo-cong-an-khong-noi-that-gia-8-5tr-th-full-noi-that-12tr-th-lh-0909-642-286-phat
6759 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-xa-lo-ha-noi/cho-thue-chung-cu-duc-khai-3-phong-ngu-day-du-noi-that-view-ngoai-thoang-mat
6760 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-xa-lo-ha-noi/cho-thue-can-ho-masteri-quan-2-1pn-nha-dep-gia-13-trieu-thang-call-0908-414-199
page 524
6761 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-xa-lo-ha-noi/cho-thue-can-ho-masteri-q2-noi-that-sieu-dep-view-q1-gia-16-trieu-lh-0902334038
6762 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong

6811 https://houseviet.vn/cho-thue-nha-rieng-nha-mat-pho-duong-lo-gom/nha-mat-tien-lo-gom-130m2-p8-q6-san-rong-60m2-kinh-doanh-tien-loi
6812 https://houseviet.vn/cho-thue-nha-rieng-nha-mat-pho-pho-24-2/cho-thue-gap-nha-duong-nhua-noi-bo-duong-24-p-11-q-6-gia-chi-20-trieu-thang
6813 https://houseviet.vn/cho-thue-nha-rieng-nha-mat-pho-duong-6-3/chot-hop-dong-nhanh-trong-thang-nay-duong-nhua-noi-bo-duong-6-p-13-q-6
6814 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-6/can-cho-thue-gap-can-ho-him-lam-cho-lon-quan-6-dt-97-m2-gia-10-trieu
6815 https://houseviet.vn/cho-thue-nha-rieng-nha-mat-pho-duong-an-duong-vuong-1/cho-thue-nha-mat-tien-an-duong-vuong-p-10-q-6-tphcm
6816 https://houseviet.vn/cho-thue-kho-nha-xuong-duong-hau-giang/can-cho-thue-kho-co-dien-tich-200m2-xay-dung-moi-an-ninh-dam-bao-bao-ve-truc-24-24-cont-40f
page 528
6817 https://houseviet.vn/cho-thue-nha-rieng-nha-mat-pho-duong-nguyen-van-luong/chinh-chu-cho-thue-02-can-nha-lien-ke-ngay-nga-tu-nguyen-van-luong-quan-6-quan-

6868 https://houseviet.vn/cho-thue-nha-rieng-nha-mat-pho-duong-bai-say-1/cho-the-nha-nguyen-can-tien-mo-vp-bai-say-phuong-8-quan-6-ho-chi-minh
6869 https://houseviet.vn/cho-thue-nha-rieng-nha-mat-pho-duong-tan-hoa-dong/mat-tien-5x18m-178b-tan-hoa-dong-phuong-14
6870 https://houseviet.vn/cho-thue-phong-tro-nha-tro-quan-6/cho-thue-nha-tro-phong-tro-quan-6
6871 https://houseviet.vn/cho-thue-nha-rieng-nha-mat-pho-duong-song-hanh-1/cho-thue-nha-tai-mt-duong-song-hanh-phuong-10-quan-6
6872 https://houseviet.vn/cho-thue-nha-rieng-nha-mat-pho-duong-binh-tay/can-ban-nhanh-nha-nguyen-can-mat-tien-duong-binh-tay-quan-6
6873 https://houseviet.vn/cho-thue-cua-hang-kiot-duong-tan-hoa-dong/can-sang-nhuong-lai-shop-thoi-trang-nam-mt-duong-tan-hoa-dong-q-6
6874 https://houseviet.vn/cho-thue-nha-rieng-nha-mat-pho-duong-pham-van-chi/cho-thue-nha-tai-quan-6-tp-hcm-cho-thue-4tr-th
6875 https://houseviet.vn/cho-thue-kho-nha-xuong-duong-so-9-cu-xa-dai-rada/toi-can-cho-thue-gap-nha-xuong-cu-xa-dai-ra-da-phu-l

...

7607 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-go-dua/cho-thue-gap-can-ho-chung-cu-sunview-town-dt-59m2-noi-that-dep-cuc-dep-nha-moi-thoang-mat
7608 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-go-dua/do-khong-su-dung-nen-can-cho-thue-gap-can-ho-chung-cu-sunview-town-dt-59m2-day-du-noi-that
7609 https://houseviet.vn/cho-thue-nha-rieng-nha-mat-pho-duong-hiep-binh/hot-can-cho-thue-nha-moi-8x30m-gom-4-lau-mt-duong-hiep-binh-gia-50tr-mot-thang
page 583
7610 https://houseviet.vn/cho-thue-nha-rieng-nha-mat-pho-duong-hoang-dieu-2-1/cuc-hot-cho-thue-nha-mt-hoang-dieu-2-thu-duc-dt-5x26-m-1-tret-3-lau-nha-moi-gia-37-trieu-m2
7611 https://houseviet.vn/cho-thue-can-ho-chung-cu-thu-duc/chinh-chu-can-cho-thue-can-ho-lotus-thu-duc
7612 https://houseviet.vn/cho-thue-nha-rieng-nha-mat-pho-duong-so-16-9/hot-chinh-chu-xuat-ngoai-cho-thue-gap-bt-gia-re-68-trieu-biet-thu-an-ninh-sang-trong-duong-16
7613 https://houseviet.vn/cho-thue-kho-nha-xuong-duong-ly-te-xuyen/khong-the-tin-noi-cho-thu

7662 https://houseviet.vn/cho-thue-nha-rieng-nha-mat-pho-duong-huynh-tan-phat-1/cho-thue-mat-bang-15-x-30m-mat-tien-huynh-tan-phat-phu-xuan
7663 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-le-van-luong-1/can-cho-thue-gap-can-ho-hung-phat-2pn-du-noi-that-gia-10-trieu-thang-lien-he-0909-718-569
7664 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-nguyen-huu-tho-1/cho-thue-can-ho-hoang-anh-an-tien-du-noi-that-10-trieu-thang-lh-0909-718-569-0906-378-770
7665 https://houseviet.vn/cho-thue-cua-hang-kiot-duong-nguyen-huu-tho-1/can-cho-thue-mat-bang-can-ho-shophouse-phu-hoang-anh-dt-30m2-gia-re-9-trieu-thang
7666 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-nguyen-huu-tho-1/cho-thue-phong-tro-tai-chcc-phu-hoang-anh-lk-quan-7-dt-15m2-gia-re-2-trieu-thang
7667 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-nguyen-huu-tho-1/cho-thue-ch-hoang-anh-an-tien-2pn-gia-7-5tr-3pn-gia-8tr-du-noi-that-gia-10tr-lh-0934-193-389
page 587
7668 https://houseviet.vn/cho-thue-can-ho-chung-c

7718 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-9/cho-thue-can-ho-flora-anh-dao-mat-duong-do-xuan-hop-quan-9-gia-6tr-thang
7719 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-9/cho-thue-can-ho-lien-ke-quan-2-day-du-tien-nghi-gia-chi-5-5-trieu-thang
7720 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-9/can-cho-thue-can-ho-anh-dao-mat-tien-do-xuan-hop-q-9-chi-7tr-thang-co-noi-that-don-vao-o-ngay
7721 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-9/cho-thue-can-ho-the-art-gia-tu-7tr-thang-block-picasso
page 591
7722 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-9/can-cho-thue-gap-can-3pn-lucky-dragon-88m2-day-du-noi-that-12-trieu-thang
7723 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-9/can-cho-thue-can-ho-the-art-dien-tich-66m2-ho-boi-cong-vien-phong-tap-gym-spa-co-noi-that
7724 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-9/cho-nguoi-nuoc-ngoai-thue-gap-ch-the-estern-q9-1pn-ddnt-tien-ich-ho-boi-gym-gia-10-trieu-th
7725 https://houseviet.vn/cho-thue-

7779 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-9/cho-thue-can-ho-flora-anh-dao-q9-2-pn-lau-11-gia-7-5-trieu-thang-noi-that-cao-cap
7780 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-9/cho-thue-can-ho-han-quoc-the-eastern-quan-9
page 595
7781 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-9/nhan-ky-gui-va-cho-thue-can-ho-the-art-gia-hoa-gia-tot-lien-he-chu-dau-tu
7782 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-9/cho-thue-can-ho-han-quoc-the-eastern-q9
7783 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-9/chinh-chu-cho-thue-nhanh-can-ho-flora-anh-dao-q9-gia-5-5tr-lh-0916775539
7784 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-9/cho-thue-can-ho-tdh-phuoc-long-46-51-60-70-71-va-83m2-gia-chi-tu-5-trieu-lh-0917-28-80-80
7785 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-9/cho-thue-can-ho-the-art-quan-9-can-goc-huong-ho-boi-cong-vien-gia-6-5-trieu-lh-0901434118
7786 https://houseviet.vn/cho-thue-can-ho-chung-cu-quan-9/cho-thue-can-ho-tdh-phuoc-long-day

page 599
7839 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-3-2-2/cho-thue-can-ho-the-everrich-q-11-115m2-2pn-lau-cao-view-dep-thoang-mat-gia-20tr-thang
7840 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-3-2-2/chinh-chu-cho-thue-can-ho-cao-cap-the-everrich-1-gia-cuc-tot
7841 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-3-2-2/cho-the-chung-cu-everrich-q-11-117m2-2pn-lh-minh-0906-678-328
7842 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-ly-nam-de/chinh-chu-cho-thue-nha-tai-chung-cu-tan-phuoc-plaza-quan-11-2pn-3pn
7843 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-3-2-2/cho-thue-cc-the-everrich-q11-160m2-3pn-nt-cao-cap-gia-22tr-th-lh-trung-0909-455-485
7844 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-le-dai-hanh-1/cho-thue-can-ho-chung-cu-the-flemington-q11-97m2-3pn-gia-18-tr-th-lh-trung-sdt-0909-455-485
7845 https://houseviet.vn/cho-thue-can-ho-chung-cu-duong-le-dai-hanh-1/cho-thue-chung-cu-flemington-q-11-dt-87m2-2pn-gia-17tr-th-lh-minh-0906-678-3